# Dropout

#### 請搭配教學影片服用
#### Tutorial video: https://www.youtube.com/watch?v=f2F9Xsd7KVk

#### 讀取tensorflow, numpy套件

In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
#from sklearn.cross_validation import train_test_split 舊版代碼，變成sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

#### 讀取digits Dataset

In [2]:
#load data
digits = load_digits() #載入digits數字圖片
X = digits.data #影像像素為64=8*8的數字圖片
y = digits.target #維度為10的one-hot encoding,表示0-9數字
y = LabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

#### 定義add_layer功能(添加神經層)

In [3]:
#定義add_layer功能(添加神經層)
#inputs, in_size, out_size, activation_funchion=None激勵函數
def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1,)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    # -> 加入dropout功能
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b,)
    tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

#### Input

In [4]:
#keep_prob保留多少百分比的神經元不被dropout
keep_prob = tf.placeholder(tf.float32)
#輸入data xs->placeholder, float32型態,None不規定sample數量,64=8*8輸入影像像素
xs = tf.placeholder(tf.float32,[None,64])
#輸入data ys->placeholder, float32型態,None不規定sample數量,10=對應x的y值為，維度為10的one-hot encoding
ys = tf.placeholder(tf.float32,[None,10]) 

#### 架構神經網路

In [6]:
#新增一層神經網路l1(隱藏層)
l1 = add_layer(xs, 64, 50, 'l1',activation_function=tf.nn.tanh)
#inputs=xs, in_size=8*8=64, out_size, name='l1',activation_function=tanh
#輸入數據, 輸入數據input data size, 隱藏層神經元數量, 命名這一層叫l1(TensorBoard),激勵函數=tanh

#新增一層輸出層prediction ///// add output(prediction) layer
prediction = add_layer(l1, 50, 10, 'l2', activation_function=tf.nn.softmax)
#inputs=l1, in_size, out_size, activation_function
#輸入數據  *來自l1隱藏層=xs, 輸入數據size  *隱藏層size, 神經元輸出數量, 激勵函數=softmax

In [8]:
#Loss : the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),axis=[1]))
#reduction_indice=[1]在新版本Tensorflow已被取代為axis=[1]

tf.summary.scalar('loss', cross_entropy)
# 舊版代碼tf.scalar_summary('loss', cross_entropy)
#Optimizer
train_step = tf.train.GradientDescentOptimizer(0.6).minimize(cross_entropy) #優化器 #優化函數:梯度下降 #0.6為Learning rate 

In [10]:
sess=tf.Session()
merged = tf.summary.merge_all() #舊版代碼 merged = tf.merge_all_summaries()
#記錄所有summary到Tensorboard

train_writer = tf.summary.FileWriter("logs/train", sess.graph) #舊版代碼tf.train.SummaryWriter
test_writer = tf.summary.FileWriter("logs/test", sess.graph)


init = tf.global_variables_initializer() #初始化/激活所有變量(Variable) #must have if define variable)
sess.run(init)

#### Training

In [12]:
for i in range(1000):
    sess.run(train_step, feed_dict={xs:X_train, ys:y_train, keep_prob:0.5})
    if i % 50 ==0:
        #record loss
        train_result = sess.run(merged, feed_dict={xs:X_train, ys:y_train, keep_prob:1})
        test_result = sess.run(merged, feed_dict={xs:X_test, ys:y_test, keep_prob:1})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)

In [ ]:
#!tensorboard --logdir='D:\Exercise\logs'